<a href="https://colab.research.google.com/github/terrysu64/Machine-Learning-and-Data-Science-Projects/blob/main/introduction_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP Fundamentals

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-19 14:16:51--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-19 14:16:51 (83.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

### import dataset
kaggle: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-19 14:16:54--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 172.253.62.128, 172.217.164.176, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-08-19 14:16:54 (120 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data('nlp_getting_started.zip')

### visualize text dataset

In [5]:
import pandas as pd
train_df = pd.read_csv('train.csv').sample(frac=1,
                                           random_state=42)
test_df = pd.read_csv('test.csv').sample(frac=1,
                                         random_state=42)
train_df.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
len(train_df), len(test_df)

(7613, 3263)

In [8]:
from random import randint
for _ in range(3):
  i = randint(1,7612)
  print(f"{'disaster' if train_df.target[i] else 'not disaster'}: {train_df.text[i]}")

not disaster: Swansea ?plot hijack transfer move for Southampton target Virgil van Dijk? http://t.co/NZRWDdLntp
not disaster: Leading emergency services boss welcomes new ambulance charity http://t.co/Mj2jQ2pSv6
not disaster: Beware of your temper and a loose tongue! These two dangerous weapons combined can lead a person to the Hellfire #islam!


### split data

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_df.text.to_numpy(),
                                                  train_df.target.to_numpy(),
                                                  test_size=0.1,
                                                  random_state=42)

In [10]:
train_df.head().to_numpy() #changes df => numpy arr

array([[3796, 'destruction', nan,
        'So you have a new weapon that can cause un-imaginable destruction.',
        1],
       [3185, 'deluge', nan,
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        0],
       [7769, 'police', 'UK',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        1],
       [191, 'aftershock', nan,
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        0],
       [9810, 'trauma', 'Montgomery County, MD',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        0]], dtype=object)

### converting text => numbers

In [11]:
#tokenization: word => a specific number
#embedding: representing relationships between tokens (can limit size + can be learned)

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

MAX_VOCAB=10000
MAX_LENGTH=15
vectorizer = TextVectorization(max_tokens=MAX_VOCAB, #limit to how many words in vocab
                               standardize="lower_and_strip_punctuation",
                               split="whitespace",
                               ngrams=None, #create grops of n words
                               output_mode="int",
                               output_sequence_length=MAX_LENGTH, #how long to limit tweet
                               pad_to_max_tokens=True #replace anything not in max_tokens to 0
)

In [12]:
vectorizer.adapt(X_train)

In [13]:
vectorizer(['hi there buddy'])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1663,   74,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [14]:
vocab = vectorizer.get_vocabulary()
vocab[:5] #top 5 words

['', '[UNK]', 'the', 'a', 'in']

### creating embedding layer

In [15]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=MAX_VOCAB,
                             output_dim=128,
                             input_length=MAX_LENGTH) #neural networks work well with %8==0

embedding(vectorizer("hi there buddy"))

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[-0.0424342 ,  0.04779292, -0.00178171, ...,  0.00631696,
         0.0441425 ,  0.04450573],
       [-0.00611545,  0.00254874,  0.00751308, ...,  0.04468616,
         0.04107905,  0.01403594],
       [-0.02189933, -0.0231041 , -0.03060955, ...,  0.03932399,
         0.0400644 ,  0.04009343],
       ...,
       [ 0.00387589,  0.00197949,  0.03989844, ...,  0.04256734,
         0.04867944, -0.01775784],
       [ 0.00387589,  0.00197949,  0.03989844, ...,  0.04256734,
         0.04867944, -0.01775784],
       [ 0.00387589,  0.00197949,  0.03989844, ...,  0.04256734,
         0.04867944, -0.01775784]], dtype=float32)>

## building models for the dataset
###multinomial naive bayes using TF-IDF formula

note: it is common to use no DL algos first

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [17]:
baseline_score = model.score(X_val, y_val)
baseline_score

0.7926509186351706

In [18]:
model.predict(X_val)[:5], y_val[:5]

(array([1, 1, 1, 0, 0]), array([0, 0, 1, 1, 1]))

In [19]:
from sklearn.metrics import classification_report

report = classification_report(model.predict(X_val),
                      y_val)
report

'              precision    recall  f1-score   support\n\n           0       0.93      0.75      0.83       516\n           1       0.63      0.89      0.73       246\n\n    accuracy                           0.79       762\n   macro avg       0.78      0.82      0.78       762\nweighted avg       0.83      0.79      0.80       762\n'

In [20]:
#               precision    recall  f1-score   support

#            0       0.93      0.75      0.83       516
#            1       0.63      0.89      0.73       246

#     accuracy                           0.79       762
#    macro avg       0.78      0.82      0.78       762
# weighted avg       0.83      0.79      0.80       762


model 1: RNN (LSTM)
connections between tensors form a directed or undirected graph along a temporal sequence.

In [47]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64, return_sequences=False)(x)
print(x.shape)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs,outputs,name="model_LSTM")

(None, 64)


In [48]:
model.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_9 (LSTM)               (None, 15, 64)            49408     
                                                                 
 lstm_10 (LSTM)              (None, 64)                33024     
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                        

In [49]:
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"]) 

In [50]:
history = model.fit(X_train, 
                    y_train,
                    epochs=5,
                    validation_data=(X_val, y_val))

Epoch 1/5
215/215 [==============================] - 5s 11ms/step - loss: 0.1366 - accuracy: 0.9574 - val_loss: 0.9935 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0586 - accuracy: 0.9745 - val_loss: 1.2326 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0534 - accuracy: 0.9758 - val_loss: 1.4884 - val_accuracy: 0.7585
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0498 - accuracy: 0.9769 - val_loss: 1.3881 - val_accuracy: 0.7572
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0453 - accuracy: 0.9775 - val_loss: 1.5232 - val_accuracy: 0.7533


### model 2: GRU

In [51]:
inputs = layers.Input(shape=(1,), dtype="string")
x = vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x)
x = layers.LSTM(64, return_sequences=False)(x)
print(x.shape)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs,outputs,name="model_GRU")

(None, 64)


In [52]:
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"]) 

In [53]:
history = model.fit(X_train, 
                    y_train,
                    epochs=5,
                    validation_data=(X_val, y_val))

Epoch 1/5
215/215 [==============================] - 6s 11ms/step - loss: 0.1374 - accuracy: 0.9507 - val_loss: 1.0416 - val_accuracy: 0.7454
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0513 - accuracy: 0.9765 - val_loss: 1.3953 - val_accuracy: 0.7559
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0525 - accuracy: 0.9752 - val_loss: 1.4087 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0506 - accuracy: 0.9777 - val_loss: 1.3961 - val_accuracy: 0.7546
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0422 - accuracy: 0.9813 - val_loss: 1.2915 - val_accuracy: 0.7520
